In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train_data = pd.read_csv("./train_data.csv")
train_y = pd.read_csv("./target.csv")

In [ ]:
import h2o
h2o.init(
  nthreads=-1,            ## -1: use all available threads
  max_mem_size = "8G")

In [ ]:
train_list=list(train_data.columns)[:-1]
train_1=h2o.H2OFrame(train_data)

In [ ]:
train_1['result']=h2o.H2OFrame(list(train_y['价格']))

In [ ]:
param = {
      "ntrees" : 2000
    , "max_depth" : 20
    , "learn_rate" : 0.02
    , "sample_rate" : 0.7
    , "col_sample_rate_per_tree" : 0.9
    , "min_rows" : 5
    , "seed": 4240
    , "score_tree_interval": 100
    ,"stopping_metric" :"MSE"
    ,"nfolds":8
    ,"fold_assignment":"AUTO"
    ,"keep_cross_validation_predictions" : True
    ,"booster":"dart"
}

In [ ]:
from h2o.estimators import H2OXGBoostEstimator
model_xgb = H2OXGBoostEstimator(**param)
model_xgb.train(x = train_list, y = 'result', training_frame = train_1)
param={
    "ntrees" : 1060
    , "max_depth" : 3
    , "learn_rate" : 0.02
    , "sample_rate" : 0.7
    , "col_sample_rate_per_tree" : 0.9
    , "min_rows" : 5
    , "seed": 4240
    , "score_tree_interval": 100
    , "stopping_metric" :"MSE"
    , "nfolds":8
    , "fold_assignment":"AUTO"
    , "keep_cross_validation_predictions" : True
 }

In [ ]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator  # import gbm estimator
model_gbm = H2OGradientBoostingEstimator(**param)
model_gbm.train(x = train_list, y = 'result', training_frame = train_1)

In [ ]:
from h2o.estimators import H2OStackedEnsembleEstimator
stack = H2OStackedEnsembleEstimator(model_id="ensemble11",
                                       training_frame=train_1,
                                       #validation_frame=test,
                                       base_models=[model_xgb.model_id,model_gbm.model_id],
                                    metalearner_algorithm="glm")
stack.train(x=train_list, y="result", training_frame=train_1)